In [6]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from torch import nn
import skimage as sk
from glob import glob
import re

In [7]:
image_list = glob('D://core_1//core_1_GMM_output//*.png')
target_path = 'D://core_1//Downsampled_mask//'
print (len(image_list))

5324


In [8]:
# As I used multi core processing for producing the gound truth then list of images has to be sorted accordingly
def sorting_path (path):
    core_id = int(path.split('core')[-1][0])*10000 + 1000
    img_id = int(path.split('img')[-1].split('.png')[0])
    return core_id + img_id

In [ ]:
# Every two High resolution binary output of the GMM model downsampled (averaged) to one low-resolution 
j = -1 # a numerator
for i in range (len(image_list[:])):
    print (image_list[i])
    if i % 2 == 0:
        mask1 = cv.imread(image_list[i], cv.IMREAD_GRAYSCALE)
        mask2 = cv.imread(image_list[i+1], cv.IMREAD_GRAYSCALE)
        new_mask = np.zeros((2,mask1.shape[0], mask1.shape[1]))
        new_mask [0,:,:] = mask1
        new_mask [1,:,:] = mask2
        resized_mask = sk.transform.downscale_local_mean(new_mask,(2,2,2) )
        resized_mask = resized_mask.reshape(resized_mask.shape[1],resized_mask.shape[2])
        print (resized_mask.shape)
        j+=1
        
        # to avoid segmenting the null images (fully black) that has no ice in it we have to skip them manually
        starting_num = 50 #number of black images in the begining of the list of images
        ending_num = 32 # number of black images in the end of the list of images
        
        if i > starting_num and i <(len(image_list)-ending_num):
            cv.imwrite(target_path+'DownSampled_{}.png'.format(j+20000), resized_mask)
            
        elif i > (len(image_list)-ending_num):
            cv.imwrite(target_path+'DownSampled_{}.png'.format(j+20000), np.zeros_like(resized_mask))
            
        else:
            cv.imwrite(target_path+'DownSampled_{}.png'.format(j+20000), np.zeros_like(resized_mask))